## This project is create to understand how ETL process using IBM files 

In [1]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime
from zipfile import ZipFile

## Download files using wget

In [2]:
# If the project don't have source.zip, you can uncomment it and download from IBM link below:
#!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2021-10-05 11:56:07--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: 'source.zip'

     0K ..                                                    100%  516M=0s

2021-10-05 11:56:08 (516 MB/s) - 'source.zip' saved [2707/2707]



## Extract files

In [4]:
with ZipFile('source.zip', 'r') as zipObj:
    zipObj.extractall('person') #extract all data files into person

## Set Path

In [5]:
tmp = "person_temp.tmp"
logfile = "person_log.txt"
targetfile = "person_transformed.csv"

## Extract

In [7]:
#Extract CSV file
def extract_file_csv(file_to_extract):
    dataframe = pd.read_csv(file_to_extract)
    return dataframe

In [8]:
#Extract JSON file
def extract_file_json(file_to_extract):
    dataframe = pd.read_json(file_to_extract, lines= True)
    return dataframe

In [25]:
#Extract XML file
def extract_file_xml(file_to_extract):
    dataframe = pd.DataFrame(columns=['name', 'height', 'weight'])
    tree = ET.parse(file_to_extract)
    root = tree.getroot()
    for person in root:
        name = person.find('name').text
        weight = float(person.find('weight').text)
        height = float(person.find('height').text)
        dataframe = dataframe.append({'name': name, 'weight': weight, 'height': height}, ignore_index= True)
    return dataframe

In [26]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight'])
    
    #Extract from CSV file:
    for csvfile in glob.glob('person\*.csv'):
        extracted_data = extracted_data.append(extract_file_csv(csvfile), ignore_index= True)
        
    #Extract from JSON file:
    for jsonfile in glob.glob('person\*.json'):
        extracted_data = extracted_data.append(extract_file_json(jsonfile), ignore_index= True)
    
    #Extract from XML file:
    for xmlfile in glob.glob('person\*.xml'):
        extracted_data = extracted_data.append(extract_file_xml(xmlfile), ignore_index= True)
        
    return extracted_data

## Tranform

In [27]:
#Convert inches to milimiters in height and pounds to kilograms in weight
# 1 inch = 0.0254 meters
# 1 pound = 0.45359237 kilograms
def transform(data):
    data['height'] = round(data.height * 0.0254, 2)
    data['weight'] = round(data.weight * 0.45359237, 2)
    return data

## Load

In [28]:
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

## Logging

In [29]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open('person_log.txt', 'a') as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL process

In [30]:
log('Start')

In [31]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
log("Load phase Started")
load(targetfile, transformed_data)
log("Load phase Ended")
log("End")